In [1]:
import pandas as pd 

In [2]:
from aind_data_access_api.document_db import MetadataDbClient

API_GATEWAY_HOST = "api.allenneuraldynamics.org"
DATABASE = 'metadata_index'
COLLECTION = 'data_assets'

docdb_api_client = MetadataDbClient(
   host=API_GATEWAY_HOST,
   database=DATABASE,
   collection=COLLECTION,
)
print(docdb_api_client._base_url)

https://api.allenneuraldynamics.org/v1/metadata_index/data_assets


In [3]:
aggregate = [
  {
    "$match": {
      "data_description.project_name": "NP Ultra and Psychedelics",
      "data_description.data_level": "derived"
    }
  },
  {
    "$project": {
      "name": 1,
      "subject_id": "$data_description.subject_id",
      "session_time": "$session.session_start_time", 
      "session_type": "$session.session_type", 
      "genotype": "$subject.genotype", 
      "modality": "$data_description.modality.name",
      "stimulus_epochs": "$session.stimulus_epochs.stimulus_name",   
    }
  },
  {
    "$project": {
      "name": 1,
      "subject_id": 1,
      "session_time": 1, 
      "session_type": 1,
      "genotype": 1,
      "modality" : 1,
      "stimulus_epochs": 1}
  }
]
records = docdb_api_client.aggregate_docdb_records(
    pipeline = aggregate,
)


In [4]:
df = pd.DataFrame(records)
df

,_id,name,subject_id,session_time,session_type,genotype,modality,stimulus_epochs
0,916393f2-7009-421a-bde7-bbaf95416937,ecephys_714527_2024-05-14_13-22-07_nwb_2025-08...,714527,2024-05-14T10:25:34.114677+00:00,A study on the effect of psilocybin on neocort...,Sim1-Cre_KJ18/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,[Extracellular electrophysiology],"[OptoTagging, OptoTagging, OptoTagging, RFMapp..."
1,c867ad1e-bd84-42df-b863-1244a2fcc957,ecephys_714789_2024-05-16_13-16-59_nwb_2025-08...,714789,2024-05-16T10:26:41.826355+00:00,A study on the effect of psilocybin on neocort...,Sim1-Cre_KJ18/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,[Extracellular electrophysiology],"[OptoTagging, OptoTagging, OptoTagging, RFMapp..."
2,d389dfbd-257a-4b19-b55f-e6c75208f1ec,ecephys_730049_2024-06-13_12-53-21_nwb_2025-08...,730049,2024-06-13T09:56:19.252728+00:00,A study on the effect of psilocybin on neocort...,Sim1-Cre_KJ18/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,[Extracellular electrophysiology],"[OptoTagging, OptoTagging, OptoTagging, OptoTa..."
3,8a00ee0c-c470-43c5-b24c-52c1785bee3c,ecephys_714527_2024-05-15_13-00-23_nwb_2025-08...,714527,2024-05-15T10:43:35.535979+00:00,A study on the effect of psilocybin on neocort...,Sim1-Cre_KJ18/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,[Extracellular electrophysiology],"[OptoTagging, OptoTagging, OptoTagging, RFMapp..."
4,48c07938-c41b-4cd6-b9e6-c25c0a90aedb,ecephys_714789_2024-05-17_11-47-07_nwb_2025-08...,714789,2024-05-17T09:26:22.868269+00:00,A study on the effect of psilocybin on neocort...,Sim1-Cre_KJ18/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,[Extracellular electrophysiology],"[OptoTagging, OptoTagging, OptoTagging, RFMapp..."
5,f6792a6f-e70b-4d42-b0b0-9007adc65775,ecephys_717033_2024-06-04_13-01-40_nwb_2025-08...,717033,2024-06-04T10:11:23.881625+00:00,A study on the effect of psilocybin on neocort...,Tlx3-Cre_PL56/wt;Ai167(TIT2L-ChrimsonR-tdT-ICL...,[Extracellular electrophysiology],"[OptoTagging, OptoTagging, OptoTagging, RFMapp..."
6,fa1023d7-2d57-4d3a-86b6-197f098adc08,ecephys_717033_2024-06-05_12-36-57_nwb_2025-08...,717033,2024-06-05T10:20:21.656058+00:00,A study on the effect of psilocybin on neocort...,Tlx3-Cre_PL56/wt;Ai167(TIT2L-ChrimsonR-tdT-ICL...,[Extracellular electrophysiology],"[OptoTagging, OptoTagging, OptoTagging, RFMapp..."
7,e50ee957-79a9-41f8-8d81-82e7a2b88a20,ecephys_717036_2024-06-06_12-32-42_nwb_2025-08...,717036,2024-06-06T09:42:55.233315+00:00,A study on the effect of psilocybin on neocort...,Tlx3-Cre_PL56/wt;Ai167(TIT2L-ChrimsonR-tdT-ICL...,[Extracellular electrophysiology],"[OptoTagging, OptoTagging, OptoTagging, RFMapp..."
8,479dabb8-1886-4e8a-b6f3-a1d66ada8999,ecephys_717036_2024-06-07_12-00-57_nwb_2025-08...,717036,2024-06-07T09:44:57.840625+00:00,A study on the effect of psilocybin on neocort...,Tlx3-Cre_PL56/wt;Ai167(TIT2L-ChrimsonR-tdT-ICL...,[Extracellular electrophysiology],"[OptoTagging, OptoTagging, OptoTagging, RFMapp..."
9,8ed4d34f-1af5-4c03-9e2b-4384a9d8133a,ecephys_729461_2024-06-10_12-51-54_nwb_2025-08...,729461,2024-06-10T09:57:49.515589+00:00,A study on the effect of psilocybin on neocort...,Tlx3-Cre_PL56/wt;Ai167(TIT2L-ChrimsonR-tdT-ICL...,[Extracellular electrophysiology],"[OptoTagging, OptoTagging, OptoTagging, OptoTa..."


In [7]:
df.to_csv('/data/metadata/npultra_metadata.csv', index= False)